## Finding Herschel `OBSID` for a sky source

Using the TAP interface to the Herschel Science Archive

In [32]:
import requests
from astropy.coordinates import SkyCoord
from astropy import units as u
import pandas as pd
from io import StringIO

In [80]:
#
#ra=251.572
#dec = 36.759
#c = SkyCoord(ra, dec, frame='icrs', unit='deg')
ra = "11h51m51s"
dec= "-79d21m19s"
c = SkyCoord(ra, dec, frame='icrs')
#

In [81]:
#
#
tap_url = 'http://archives.esac.esa.int/hsa/whsa-tap-server/tap/sync?REQUEST=doQuery&LANG=ADQL&FORMAT=csv&'
tap_cat = "v_active_observation"
radius = 30.0/3600.0 # 30 arcsec circle
circle = f"circle('ICRS',{c.ra.value:.4f},{c.dec.value:.4f},{radius:.4f})"
#dist = f"DISTANCE(POINT('ICRS',t.ra,t.dec),POINT('ICRS',{c.ra.value:.4f},{c.dec.value:.4f}))"
# 
print (f"Search in {tap_cat} for observations that intersect with {circle}")
#
query = f"QUERY=select top 100 t.observation_id,t.od_number,t.target_name,t.ra,t.dec,t2.observing_mode_name" + \
    " from v_active_observation as t, observing_mode as t2 " + \
    " WHERE " + \
    " (t.observing_mode_oid=t2.observing_mode_oid) and " + \
    f"(1=intersects(polygon_fov,{circle}))"
xreq = tap_url + query
with requests.get(xreq) as r:
    jx = r.content.decode()
#
if ("ERROR" in jx):
    print ("The query resulted in an error, please correct.")
    raise Exception
else:
    buff = StringIO(jx)
    df = pd.read_csv(buff, sep=",")
    nfound = df.shape[0]
    print (f"Found {nfound} observations")

Search in v_active_observation for observations that intersect with circle('ICRS',177.9625,-79.3553,0.0083)
Found 6 observations


In [82]:
#
df

,observation_id,od_number,target_name,ra,dec,observing_mode_name
0,1342211293,583,DC300-17,177.542254,-79.383009,SpirePacsParallel
1,1342193045,320,DC300-17-c2,178.244806,-79.386568,PacsPhoto
2,1342193044,320,DC300-17-c2,178.244806,-79.386568,PacsPhoto
3,1342212300,600,DC300-17,177.542254,-79.383009,SpirePacsParallel
4,1342211293,583,DC300-17,178.503108,-79.378926,SpirePacsParallel
5,1342212300,600,DC300-17,178.508450,-79.379248,SpirePacsParallel


In [83]:
inst_select = "PacsPhoto"
if (nfound > 0):
    out_df = df.query(f"observing_mode_name == '{inst_select}'")
    nout = out_df.shape[0]

In [85]:
#
print (f"{c.ra.value:.5f},{c.dec.value:.5f},{nout},{inst_select},",end="")
for j in range(nout):
    print (f"{out_df['observation_id'].iloc[j]:10},",end="")
print()

177.96250,-79.35528,2,PacsPhoto,1342193045,1342193044,


1342225443